In [1]:
from neo import NeoGraph
from db import Db

import pandas as pd

g = NeoGraph()

db = Db()
eng = db.create_engine()
df = pd.read_sql("""select * from hsk5k""", eng)

In [2]:
df['len'] = df['characters'].apply(lambda x: len(list(x)))
df['c1'] = df['characters'].apply(lambda x: list(x)[0])
df['c2'] = df['characters'].apply(lambda x: list(x)[1] if len(x) >= 2 else None)
df['p1'] = df['pinyin'].apply(lambda x: x.split(' ')[0])
df['p2'] = df['pinyin'].apply(lambda x: x.split(' ')[1] if ' ' in x else None)
df['english'] = df['english'].apply(lambda x: x.replace("'", "\\'"))

In [3]:
df

,hsk,characters,pinyin,english,len,c1,c2,p1,p2
0,3,阿姨,a1 yi2,maternal aunt; step-mother; childcare worker; ...,2,阿,姨,a1,yi2
1,3,啊,a5,"modal particle ending sentence, showing affirm...",1,啊,None,a5,None
2,5,唉,ai1,interjection or grunt of agreement or recognit...,1,唉,None,ai1,None
3,6,哎哟,ai1 yo1,hey; ow; ouch; interjection of pain or surprise,2,哎,哟,ai1,yo1
4,6,挨,ai2,to suffer from; to endure; to tide over (a dif...,1,挨,None,ai2,None
...,...,...,...,...,...,...,...,...,...
4997,6,作息,zuo4 xi1,work and rest,2,作,息,zuo4,xi1
4998,3,作业,zuo4 ye4,school assignment; homework; work; task; opera...,2,作,业,zuo4,ye4
4999,3,作用,zuo4 yong4,to act on; to affect; action; function; activi...,2,作,用,zuo4,yong4
5000,6,座右铭,zuo4 you4 ming2,motto; maxim,3,座,右,zuo4,you4


In [4]:
df[df['p1'] == 'fen1']

,hsk,characters,pinyin,english,len,c1,c2,p1,p2
1081,3,分,fen1,to divide; to separate; to allocate; to distin...,1,分,None,fen1,None
1082,6,分辨,fen1 bian4,to distinguish; to differentiate; to resolve; ...,2,分,辨,fen1,bian4
1083,5,分别,fen1 bie2,to part or leave each other; to distinguish; d...,2,分,别,fen1,bie2
1084,5,分布,fen1 bu4,distributed; to distribute,2,分,布,fen1,bu4
1085,6,分寸,fen1 cun5,propriety; appropriate behavior; proper speech...,2,分,寸,fen1,cun5
1086,5,纷纷,fen1 fen1,one after another; in succession; one by one; ...,2,纷,纷,fen1,fen1
1087,6,吩咐,fen1 fu4,to tell; to instruct; to command,2,吩,咐,fen1,fu4
1088,6,分红,fen1 hong2,a bonus; to award a bonus,2,分,红,fen1,hong2
1089,6,分解,fen1 jie3,to resolve; to decompose; to break down,2,分,解,fen1,jie3
1090,6,分裂,fen1 lie4,to split up; to divide; to break up; fission,2,分,裂,fen1,lie4


In [5]:
df.groupby(['c1','p1']).size().reset_index().sort_values(by = 'c1')['c1'].value_counts().head(50)

兴    3
重    3
得    3
长    3
着    3
倒    2
埋    2
亚    2
要    2
划    2
角    2
为    2
还    2
干    2
大    2
乐    2
应    2
会    2
中    2
分    2
传    2
挑    2
强    2
几    2
散    2
切    2
盛    2
元    2
曲    2
好    2
折    2
振    2
抢    2
似    2
地    2
了    2
正    2
称    2
薄    2
差    2
国    2
都    2
调    2
恶    2
漂    2
给    2
别    2
朝    2
模    2
答    2
Name: c1, dtype: int64

In [6]:
df[df['c1'] == '着']

,hsk,characters,pinyin,english,len,c1,c2,p1,p2
4602,3,着急,zhao2 ji2,to worry; to feel anxious,2,着,急,zhao2,ji2
4603,5,着凉,zhao2 liang2,catch cold,2,着,凉,zhao2,liang2
4604,6,着迷,zhao2 mi2,to be fascinated; to be captivated,2,着,迷,zhao2,mi2
4623,2,着,zhe5,particle attached after verb to indicate actio...,1,着,None,zhe5,None
4891,6,着手,zhuo2 shou3,to put one\'s hand to it; to start out on a ta...,2,着,手,zhuo2,shou3
4892,6,着想,zhuo2 xiang3,to give thought (to others); to consider (othe...,2,着,想,zhuo2,xiang3
4894,6,着重,zhuo2 zhong4,put emphasis on; to stress; to emphasize,2,着,重,zhuo2,zhong4


In [7]:
helper = []
chars = []
for i, c in df.iterrows():
    for h, p in zip(list(c['characters']), c['pinyin'].split(' ')):
        helper.append(f'{h} : {p}')
        chars.append(h)
        

In [8]:
from collections import Counter, OrderedDict

In [9]:
g.truncate()

----- Truncating graph -----


In [10]:
g.add_characters(list(set(chars)))

  0%|          | 0/2634 [00:00<?, ?it/s]

----- Adding chars -----


100%|██████████| 2634/2634 [00:02<00:00, 1042.42it/s]

----- Done -----


In [11]:
g.create_links(df[df['len'] == 2])

  0%|          | 7/4025 [00:00<01:00, 66.07it/s]

----- Linking two character words -----


100%|██████████| 4025/4025 [00:23<00:00, 172.72it/s]

----- Done -----
